### Setup

1. Create AI Search Connection
2. Update AI_SEARCH_CONNECTION_NAME
3. Update AI_SEARCH_INDEX_NAME

### Connect to Foundry project

In [1]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise



In [2]:
import os
from azure.monitor.opentelemetry import configure_azure_monitor

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = project.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

In [3]:
from azure.ai.inference.tracing import AIInferenceInstrumentor 

# Instrument AI Inference API 

AIInferenceInstrumentor().instrument()

In [4]:
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType
import sys
connection = project.connections.get(connection_name=os.environ["AI_SEARCH_CONNECTION_NAME"])
conn_id = connection.id
indexName = os.environ["AI_SEARCH_INDEX_NAME"]
print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=conn_id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run
project.telemetry.enable(destination=sys.stdout)

agent = project.agents.get_agent(
    agent_id="asst_9Tq1e6fHBqdhlXcRRdttiXWK"
)


"""agent = project.agents.create_agent(
    model=os.getenv("CHAT_MODEL"),
    name="AISearchAgent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)"""

/subscriptions/382e9d43-0c24-4bf6-b807-0a4935bdc6f6/resourceGroups/aidemo2/providers/Microsoft.MachineLearningServices/workspaces/aga-demo2/connections/genaisimcogsrch


'agent = project.agents.create_agent(\n    model=os.getenv("CHAT_MODEL"),\n    name="AISearchAgent",\n    instructions="You are a helpful assistant",\n    tools=ai_search.definitions,\n    tool_resources=ai_search.resources\n)'

### Create a thread for conversation presistance

In [5]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_dhQA7JKImpxgTXIf7eF32tsu


In [6]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


### Get the response

In [7]:
sendMessage("Do you have Reid's SEC filling?")

Created message, ID: msg_lXOZVoGfRsI4KaTh5PGFJNab
Run finished with status: RunStatus.COMPLETED
Assistant: Yes, I have located Reid's SEC filing. Here is some information from the filing:

- Reporting Person: Reid Hoffman
- Address: C/O Microsoft Corporation, One Microsoft Way, Redmond, WA 98052-6399
- Issuer Name and Ticker: Microsoft Corp [MSFT]
- Earliest Transaction Date: March 13, 2025
- Non-Derivative Securities Owned: Common stock, 15,905 shares indirectly by Living Trust
- Derivative Securities Owned: Restricted Stock Units, 34.493 shares of common stock

For further detailed information, you can refer to the document titled `0001062993-25-005509.pdf`【3:0†source】.


In [8]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  azure_ad_token_provider=token_provider,
  api_version="2024-10-21"
)
# from openai import AzureOpenAI
# client = AzureOpenAI()

completion = client.chat.completions.create(
  model="gpt-4o", # Replace with your model dpeloyment name.
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "When was Microsoft founded?"}
  ]
)

#print(completion.choices[0].message)
print(completion.model_dump_json(indent=2) )


{
  "id": "chatcmpl-BWlXFWqLCfnoVfgBi2a8fI2BeLemo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Microsoft was founded on **April 4, 1975** by **Bill Gates** and **Paul Allen** in Albuquerque, New Mexico.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1747148921,
  "model": "gpt-4o",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint"